In [39]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [40]:
!pip install langchain langchain-community sentence-transformers -q

In [41]:
!pip install pdfminer.six -q

In [42]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from pdfminer.high_level import extract_text
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import paired_euclidean_distances
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
doc1_text = extract_text("doc1.pdf")
doc2_text = extract_text("doc2.pdf")
doc3_text = extract_text("doc3.pdf")

In [44]:
print(doc1_text)

Large language models, also known as LLMs, are very large deep
learning models that are pre-trained on vast amounts of data. The
underlying transformer is a set of neural networks that consist of an
encoder and a decoder with self-attention capabilities. The encoder and
decoder extract meanings from a sequence of text and understand the
relationships between words and phrases in it.

Transformer LLMs are capable of unsupervised training, although a more
precise explanation is that
is
through this process that transformers learn to understand basic grammar,
languages, and knowledge.

transformers perform self-learning. It

Unlike earlier recurrent neural networks (RNN) that sequentially process
inputs, transformers process entire sequences in parallel. This allows the
data scientists to use GPUs for
training transformer-based LLMs,
significantly reducing the training time.

arge language models are incredibly flexible. One model can perform
completely different tasks such as answering q

In [45]:
print(doc2_text)

Name : Amey Kelkar
Address : AB - 406, Signature residency, Kolar road, Bhopal, 462042.

LangChain is a framework for developing applications powered by large
language models (LLMs).

LangChain simplifies every stage of the LLM application lifecycle:

blocks,

third-party

components,

Development: Build your applications using LangChain's open-source
integrations. Use
and
building
LangGraph to build stateful agents with first-class streaming and human-
in-the-loop support.
Productionization: Use LangSmith to inspect, monitor and evaluate your
chains, so that you can continuously optimize and deploy with confidence.
Deployment: Turn your LangGraph applications into production-ready
APIs and Assistants with LangGraph Cloud.

Concretely, the framework consists of the following open-source libraries:

langchain-core: Base abstractions and LangChain Expression Language.
langchain-community: Third party integrations.
Partner packages (e.g. langchain-openai, langchain-anthropic, etc.): Some


In [46]:
print(doc3_text)

Torrey Canyon oil spill

The Torrey Canyon oil spill was one of the world's most serious oil spills.
The supertanker SS Torrey Canyon ran aground on rocks off the south-
west coast of the United Kingdom in 1967, spilling an estimated 25–36
million gallons (94–164 million litres) of crude oil.[1] Attempts to
mitigate the damage included the bombing of the wreck by aircraft from
the Royal Navy and Royal Air Force. Hundreds of miles of coastline in
Britain, France, Guernsey, and Spain were affected by the oil and other
substances used to mitigate damage.[2] It was the world's worst oil spill
and led to significant changes in maritime law and oil spill responses.

Background
Main article: SS Torrey Canyon
When laid down in the United States in 1959, Torrey Canyon had a
capacity of 60,000 tons; the ship was later enlarged to 120,000 tons in
Japan. [3] She was named for a geographical feature in California. Torrey
Canyon was registered in Liberia and owned by Barracuda Tanker
Corporation, a 

In [47]:
def retrieve(doc, query):
  vec = TfidfVectorizer()
  vec_doc = vec.fit_transform(doc)
  vec_query = vec.transform([query])
  similarity = cosine_similarity(vec_query, vec_doc)
  index = similarity.argmax()
  print(f"retrieved document :  {doc[index]}\n index : {index}")
  print(f"similarity score : {similarity[0][index] * 100}")

  return doc[index]

def simple_preprocess(text):
  lm = WordNetLemmatizer()
  sentences = sent_tokenize(text)
  preprocessed = []
  for i in range(len(sentences)):
    x = re.sub("[^a-zA-Z0-9]", " ", sentences[i])
    x = x.lower().split()
    x = [lm.lemmatize(word) for word in x if word not in set(stopwords.words("english"))]
    x = " ".join(x)
    preprocessed.append(x)

  return preprocessed


In [48]:
preprocessed_doc1 = simple_preprocess(doc1_text)
preprocessed_doc2 = simple_preprocess(doc2_text)
preprocessed_doc3 = simple_preprocess(doc3_text)

In [49]:
query = 'name address'

In [50]:
doc1_r = retrieve(preprocessed_doc1, query)

retrieved document :  name amey kelkar address ab 406 signature residency kolar road bhopal 462042
 index : 21
similarity score : 40.824829046386306


In [51]:
doc2_r = retrieve(preprocessed_doc2, query)

retrieved document :  name amey kelkar address ab 406 signature residency kolar road bhopal 462042
 index : 0
similarity score : 40.824829046386306


In [52]:
doc3_r = retrieve(preprocessed_doc3, query)

retrieved document :  name amey kelkar address ab 406 signature residency kolar road bhopal 462042
 index : 71
similarity score : 40.824829046386306


In [53]:
def jaccard_similarity(sentence1, sentence2):
    # Convert sentences to sets of words
    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())

    # Calculate intersection and union
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Return Jaccard similarity
    return (len(intersection) / len(union)) * 100 if len(union) > 0 else 0

# Two sentences
sentence1 = "Python is a programming language."
sentence2 = "Python is widely used for programming."

# Calculate Jaccard similarity
similarity_score = jaccard_similarity(sentence1, sentence2)
print(f"Jaccard Similarity: {similarity_score}")

Jaccard Similarity: 22.22222222222222


In [54]:
import pandas as pd
def cosine_similar_sent(sent1, sent2):
  vec = TfidfVectorizer()
  mat = vec.fit_transform([sent1, sent2])
  similarity = cosine_similarity(mat[0:1], mat[1:2])
  return similarity[0][0] * 100

def euclid_dis(sent1, sent2):
  vec = TfidfVectorizer()
  mat = vec.fit_transform([sent1, sent2])
  similarity = paired_euclidean_distances(mat[0:1], mat[1:2])
  return similarity[0]


def jaccard_similarity(sentence1, sentence2):
    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    return (len(intersection) / len(union)) * 100 if len(union) > 0 else 0

In [55]:
score1 = [jaccard_similarity(doc1_r, doc2_r), jaccard_similarity(doc1_r, doc3_r)]
score2 = [cosine_similar_sent(doc1_r, doc2_r), cosine_similar_sent(doc1_r, doc3_r)]
score3 = [euclid_dis(doc1_r, doc2_r), euclid_dis(doc1_r, doc3_r)]
score2

[100.00000000000003, 100.00000000000003]

In [56]:
scores = pd.DataFrame(
    columns = ['doc1_to_doc2', 'doc1_to_doc3'],
    data = [score1, score2, score3],
    index = ['jaccard similarity', 'cosine similarity', 'euclidean distance']
    )


In [57]:
scores

,doc1_to_doc2,doc1_to_doc3
jaccard similarity,100.0,100.0
cosine similarity,100.0,100.0
euclidean distance,0.0,0.0
